In [11]:
import pandas as pd
from Bio import SeqIO
from Bio.Alphabet import generic_protein
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import Pipeline
from sklearn.manifold import TSNE
from scipy.cluster.hierarchy import fcluster, to_tree


import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from covid_lib import *

from scipy.spatial.distance import pdist
from scipy.cluster.hierarchy import linkage, dendrogram
from gensim.models import Word2Vec
from sklearn.model_selection import ParameterGrid

from ete3 import Tree

## data - merge sequence data with meta

In [2]:
orf1ab_seqs = list(SeqIO.parse('../../master/data/coronavirus_orf1ab.fasta','fasta',alphabet=generic_protein))
orf1ab_meta = pd.read_csv("../../master/data/coronavirus_orf1ab_meta.csv")

clustal_tree = "../orf1ab_clustal_tree.out"

In [3]:
d={ 'Accession':[], 'Sequence':[]}
for s in orf1ab_seqs:
    d['Accession'].append(s.name.strip())
    d['Sequence'].append(str(s.seq))

df_seq = pd.DataFrame(d)

df_seq = df_seq.drop_duplicates()

df_seq = df_seq[df_seq.Sequence.apply(lambda s: not s.startswith("L"))].copy()

In [4]:
orf1ab_meta = orf1ab_meta.drop_duplicates()
orf1ab_meta

,Accession,Release_Date,Species,Length,Geo_Location,Host,Isolation_Source,Collection_Date,GenBank_Title
0,YP_009724389,2020-01-13T00:00:00Z,Severe acute respiratory syndrome-related coro...,7096,China,Homo sapiens,NaN,2019-12,orf1ab polyprotein [Severe acute respiratory s...
1,YP_009555238,2019-02-21T00:00:00Z,Betacoronavirus 1,7095,USA,NaN,NaN,NaN,Orf1ab [Human coronavirus OC43]
2,YP_002308478,2018-08-24T00:00:00Z,Bulbul coronavirus HKU11,6264,Hong Kong,Pycnonotus jocosus,NaN,2007-01,orf1ab polyprotein [Bulbul coronavirus HKU11-934]
3,YP_009513008,2018-08-24T00:00:00Z,Hedgehog coronavirus 1,7150,Germany,Erinaceus europaeus,feces,2012,orf1ab [Betacoronavirus Erinaceus/VMC/DEU/2012]
4,YP_009513020,2018-08-24T00:00:00Z,Coronavirus HKU15,6267,China: Hong Kong,Sus scrofa,NaN,2010,replicase polyprotein [Porcine coronavirus HKU15]
...,...,...,...,...,...,...,...,...,...
2711,QIU78777,2020-04-06T00:00:00Z,Severe acute respiratory syndrome-related coro...,7096,Spain,Homo sapiens,NaN,2020-03-10,ORF1ab polyprotein [Severe acute respiratory s...
2712,QIU78717,2020-04-06T00:00:00Z,Severe acute respiratory syndrome-related coro...,7096,Spain,Homo sapiens,NaN,2020-03-10,ORF1ab polyprotein [Severe acute respiratory s...
2713,QIU78705,2020-04-06T00:00:00Z,Severe acute respiratory syndrome-related coro...,7096,Spain,Homo sapiens,NaN,2020-03-09,ORF1ab polyprotein [Severe acute respiratory s...
2714,QIU78741,2020-04-06T00:00:00Z,Severe acute respiratory syndrome-related coro...,7096,Spain,Homo sapiens,NaN,2020-03-10,ORF1ab polyprotein [Severe acute respiratory s...


In [5]:
df = df_seq.merge(orf1ab_meta, on='Accession', how='left')


In [6]:
### new columns for easy access
df['Protein'] = df.GenBank_Title.apply(lambda s: s.split("[")[0].strip())
df[df.Protein == 'orf1ab polyprotein, partial']

df['Country'] = df.Geo_Location.apply(lambda s: s.split(':')[0] if isinstance(s, str) else s )

df['Host_agg'] = df.Host.apply(lambda s: s.split(" ")[0] if isinstance(s, str) else s )

## build corpus for word2vec and create lists of labels

In [8]:
proteomes = df.Sequence.copy()

kmer_len = 3

#let's keep the original indexing to allow lookups from the starting data
corpus = []
ordered_indexes = [] #probably overkill, but let's avoid forgetting the indexes to retrieve metadata

for i, seq in proteomes.iteritems():
    ordered_indexes.append(i)
    corpus.append([seq[j:j+kmer_len] for j in np.arange(0, len(seq)-kmer_len)])
    
#All that is required is that the input yields one sentence (list of utf8 words) after another.

#labels preparation
geo_labels = []
date_labels = []
host_labels = []
host_agg_labels = []
gen_bank_labels = []
accession_labels = []
for i, sentence in zip(ordered_indexes, corpus):
    accession_labels.append(df.iloc[i]['Accession'])
    geo_labels.append(df.iloc[i]['Country'])
    date_labels.append(df.iloc[i]['Release_Date'])
    host_labels.append(df.iloc[i]['Host'])
    host_agg_labels.append(df.iloc[i]['Host_agg'])
    gen_bank_labels.append(df.iloc[i]['GenBank_Title'])

In [ ]:
param_grid = {
    'size' : [10, 50, 100, 200, 300],
    'iter' : [100, 200],
    'method': ['ward',],
    'metric': ['cosine',],
}

labs = []
results = pd.DataFrame(np.ones((len(param_grid['size']), len(param_grid['iter']))), index=param_grid['size'],
             columns=param_grid['iter'])

for params in ParameterGrid(param_grid):
    print(params)
    
    #train w2v model
    model = Word2Vec(sentences=corpus, min_count=1, size=params['size'], iter=params['iter'], workers=8)

    model.save("models/size_{}_iter_{}.model".format(params['size'],params['iter']))
    
    #get embedded sequence vectors
    vecs=[]
    for i, sentence in zip(ordered_indexes, corpus):
        vecs.append(np.mean([model.wv[word] for word in sentence], axis=0))

    #distance and linkage matrices
    dm = pdist(vecs, params['metric'])
    Z = linkage(dm, method=params['method'], metric=params['metric'], optimal_ordering=False)
    
    #tree comparison
    leaf_names=accession_labels
    tree = to_tree(Z,False)
    t1 = Tree(getNewick(tree, "", tree.dist, leaf_names))
    t2 = Tree(clustal_tree)
    rf= t1.robinson_foulds(t2)
    #print(t1, t2)
    rf_ratio = rf[0]/rf[1]
    
    results.loc[params['size'],params['iter']] = rf_ratio
    
    print("RF distance is {} over a total of {} = {}".format(rf[0], rf[1], rf[0]/rf[1]))
    #print("Partitions in tree2 that were not found in tree1:", rf[3] - rf[4])
    #print("Partitions in tree1 that were not found in tree2:", rf[4] - rf[3])
    ############################################################
    
results.to_csv("data/orf1ab_AA_rf_clustal_size_iter.csv")

{'iter': 100, 'method': 'ward', 'metric': 'cosine', 'size': 10}
RF distance is 3372 over a total of 5424 = 0.6216814159292036
{'iter': 100, 'method': 'ward', 'metric': 'cosine', 'size': 50}
RF distance is 2748 over a total of 5424 = 0.5066371681415929
{'iter': 100, 'method': 'ward', 'metric': 'cosine', 'size': 100}
RF distance is 2526 over a total of 5424 = 0.4657079646017699
{'iter': 100, 'method': 'ward', 'metric': 'cosine', 'size': 200}
